In [2]:
import pandas as pd
import numpy as np
import multiprocessing as mp

# #1st dataset
df_train_labels_1 = pd.read_csv("advanced-learning-models-2020/Ytr0.csv")
train_labels_1 = np.ravel(df_train_labels_1['Bound'].to_numpy())

# file_train_feat_mat_1 = open("advanced-learning-models-2020/Xtr0_mat100.csv")
# train_feat_mat_1 = np.loadtxt(file_train_feat_mat_1, delimiter=" ")

# file_test_feat_mat_1 = open("advanced-learning-models-2020/Xte0_mat100.csv")
# test_feat_mat_1 = np.loadtxt(file_test_feat_mat_1, delimiter=" ")

# #2nd dataset 
df_train_labels_2 = pd.read_csv("advanced-learning-models-2020/Ytr1.csv")
train_labels_2 = np.ravel(df_train_labels_2['Bound'].to_numpy())

# file_train_feat_mat_2 = open("advanced-learning-models-2020/Xtr1_mat100.csv")
# train_feat_mat_2 = np.loadtxt(file_train_feat_mat_2, delimiter=" ")

# file_test_feat_mat_2 = open("advanced-learning-models-2020/Xte1_mat100.csv")
# test_feat_mat_2 = np.loadtxt(file_test_feat_mat_2, delimiter=" ")

# #3rd dataset
df_train_labels_3 = pd.read_csv("advanced-learning-models-2020/Ytr2.csv")
train_labels_3 = np.ravel(df_train_labels_3['Bound'].to_numpy())

# file_train_feat_mat_3 = open("advanced-learning-models-2020/Xtr2_mat100.csv")
# train_feat_mat_3 = np.loadtxt(file_train_feat_mat_3, delimiter=" ")

# file_test_feat_mat_3 = open("advanced-learning-models-2020/Xte2_mat100.csv")
# test_feat_mat_3 = np.loadtxt(file_test_feat_mat_3, delimiter=" ")


##### RAW DATA

#1st dataset
Xtr0 = pd.read_csv("advanced-learning-models-2020/Xtr0.csv")
Xtr0=Xtr0['seq'].tolist()

Xte0 = pd.read_csv("advanced-learning-models-2020/Xte0.csv")
Xte0=Xte0['seq'].tolist()

#2nd dataset
Xtr1 = pd.read_csv("advanced-learning-models-2020/Xtr1.csv")
Xtr1=Xtr1['seq'].tolist()

Xte1 = pd.read_csv("advanced-learning-models-2020/Xte1.csv")
Xte1=Xte1['seq'].tolist()

#3rd dataset
Xtr2 = pd.read_csv("advanced-learning-models-2020/Xtr2.csv")
Xtr2=Xtr2['seq'].tolist()

Xte2 = pd.read_csv("advanced-learning-models-2020/Xte2.csv")
Xte2=Xte2['seq'].tolist()


In [3]:
def export_predict(pred_1,pred_2,pred_3,file_name):
    df_final_results_ridge = pd.DataFrame({'Id':np.array(range(3000)),'Bound':np.concatenate([pred_1,pred_2,pred_3])})
    df_final_results_ridge.to_csv(file_name,index=False)

## Ridge Regression

In [13]:
#rescaling
def rescale_zero_minus_one(n):
    if n==0:
        return -1
    return n

def rescale_vector_zero_minus_one(y):
    return np.array(list(map(rescale_zero_minus_one,train_labels_1)))

def rescale_minus_one_zero(n):
    if n==-1:
        return 0
    return n

def rescale_vector_zero_minus_one(y):
    return np.array(list(map(rescale_minus_one_zero,train_labels_1)))


#Ridge without kernel
def ridgeEstimators(train_fm,train_lm,lam):
    a = np.matmul(train_fm.T,train_fm)
    b = lam*np.identity(len(train_fm[0]))
    c = np.linalg.inv(a+b)
    d = np.matmul(train_fm.T,train_lm)
    return np.matmul(c,d)

def round_zero_one(pred_val):
    if pred_val<0:
        return 0
    if pred_val>1:
        return 1
    return (round(pred_val))

def predictRidge(X,estimators):
    result = map(round_zero_one, np.matmul(X,estimators)) 
    return np.array(list(result)) 

#ridge predictions:
# estimators_1 = ridgeEstimators(train_feat_mat_1,train_labels_1,0.001)
# predictionsRidge_1 = predictRidge(test_feat_mat_1,estimators_1)

# estimators_2 = ridgeEstimators(train_feat_mat_2,train_labels_2,0.001)
# predictionsRidge_2 = predictRidge(test_feat_mat_2,estimators_2)

# estimators_3 = ridgeEstimators(train_feat_mat_3,train_labels_3,0.001)
# predictionsRidge_3 = predictRidge(test_feat_mat_3,estimators_3)

## KERNEL Ridge Regression

In [12]:
################################### KERNELS ###################################

###GAUSSIAN KERNEL
def gaussian_kernel(x,y):
    sig = 5
    diff = x-y 
    return np.exp(-np.dot(diff,diff)/2*sig*sig)

###SPECTRUM KERNEL
def spectrum_kernel(x1,x2,k=6):
    dic1=dict()
    dic2=dict()
    for i in range(len(x1)-k+1):
        k_mer_1 = x1[i:k+i]
        if k_mer_1 not in dic1:
            dic1[k_mer_1]=1
        else:
            dic1[k_mer_1]+=1
        k_mer_2 = x2[i:k+i]
        if k_mer_2 not in dic2:
            dic2[k_mer_2]=1
        else:
            dic2[k_mer_2]+=1
    common=0
    for j in dic1:
        if j in dic2:
            common+=dic1[j]*dic2[j]
    return common

###MISMATCH KERNEL
def count_equal(x1,x2):
    count = 0
    for i in range(len(x1)):
        if x1[i]==x2[i]:
            count+=1
    return count

def mismatch_kernel(x1,x2):
    k=8
    num_mis=2
    count_common_k_mer = 0
    for i in range(len(x1)-k+1):
        k_mer_1 = x1[i:k+i]
        for j in range(len(x2)-k+1):
            k_mer_2 = x2[j:k+j]
            if k_mer_1==k_mer_2:
                count_common_k_mer+=1
            else:
                count_comm_seq_mon = count_equal(k_mer_1,k_mer_2) #number of common monomers taking the seq into account
                if (count_comm_seq_mon >=k-num_mis):
                    count_common_k_mer+= np.exp(count_comm_seq_mon/k -1)*(count_comm_seq_mon/k)*(count_comm_seq_mon/k)
    return count_common_k_mer
    
############################### K MATRICES ####################################


### not symmetric K matrix -> test time
def compute_gram_matrix(kernel,X1,X2):
    N1=len(X1)
    N2=len(X2)
    gram_matrix = np.zeros(shape=(N1,N2))
    for i in range(N1):
        for j in range(N2):
            gram_matrix[i][j]=kernel(X1[i],X2[j])
    return gram_matrix

### not symmetric K matrix -> test time: parallel computing version 
def compute_gram_matrix_parallel(kernel,X1,X2):
    N1=len(X1)
    N2=len(X2)
    gram_matrix = np.zeros(shape=(N1,N2))
    for i in range(N1):
        #paralellize 
        pool = mp.Pool(mp.cpu_count())
        results = [pool.apply(kernel, args=(X1[i],xj)) for xj in X2]
        pool.close()   
        gram_matrix[i]=results
    return gram_matrix


### symmetric K matrix -> train time: parallel computing version
def compute_gram_matrix_train(kernel,X):
    N=len(X)
    gram_matrix = np.zeros(shape=(N,N))
    for i in range(0,N):
        #paralellize 
        pool = mp.Pool(mp.cpu_count())
        results = [pool.apply(kernel, args=(X[i],xj)) for xj in X[i:N]]
        pool.close()    
        for j in range(i,N):
            gram_matrix[i][j]=gram_matrix[j][i]=results[j-i] 
    return gram_matrix




################################################ KRR #################################

def kernelRidgeEstimators(gram_matrix,train_lm,lam):
    N=len(gram_matrix)
    a = lam*N*np.identity(N)
    b = np.linalg.inv(gram_matrix+a)
    return np.matmul(b,train_lm)


In [7]:
def spectrum_combination(x1,x2,k_list):
    return np.sum((1/len(k_list))*np.array([spectrum_kernel(x1,x2,k) for k in k_list]))

def wrap_kernel_0(x1,x2):
    return spectrum_combination(x1,x2,[3,5,6,10]) 

def wrap_kernel_1(x1,x2):
    return spectrum_combination(x1,x2,[4,6,10])

def wrap_kernel_2(x1,x2):
    return spectrum_combination(x1,x2,[2,6,15])

In [8]:
def split(X,Y):
    return X[:1500],Y[:1500],X[1500:],Y[1500:]

def evaluate(Y_label,Y_pred):
    return np.sum(Y_label==Y_pred)/len(Y_label)

In [9]:
#using parallelized versions
def fit_KRR(X_train,y_train,kernel,lam): #return KRR estimators to be used in predict_KRR
    train_gram = compute_gram_matrix_train(kernel,X_train)
    return train_gram,kernelRidgeEstimators(train_gram,y_train,lam)

def predict_KRR(X_train,X_test,kernel,estimators):
    test_K_mat = compute_gram_matrix_parallel(kernel,X_test,X_train) #analog to gram matrix but not symmetric
    return test_K_mat,predictRidge(test_K_mat,estimators)

In [11]:
lam_0=0.1
lam_1=0.15
lam_2=0.02

In [ ]:
#final predictions
le_train_gram_0,le_estim_0=fit_KRR(Xtr0,train_labels_1,wrap_kernel_0,lam_0)
le_test_gram_0,le_predic_0=predict_KRR(Xtr0,Xte0,wrap_kernel_0,le_estim_0)

le_train_gram_1,le_estim_1=fit_KRR(Xtr1,train_labels_2,wrap_kernel_1,lam_1)
le_test_gram_1,le_predic_1=predict_KRR(Xtr1,Xte1,wrap_kernel_1,le_estim_1)

le_train_gram_2,le_estim_2=fit_KRR(Xtr2,train_labels_3,wrap_kernel_2,lam_2)
le_test_gram_2,le_predic_2=predict_KRR(Xtr2,Xte2,wrap_kernel_2,le_estim_2)

In [ ]:
export_predict(le_predic_0,le_predic_1,le_predic_2,'sev_linear_comb_spect_kernel_sev_lam.csv')

#### assessing accuracy for some hyperparameter values

In [ ]:
#splits to assess value of hyperparameters
X_tr_0,Y_tr_0,X_ts_0,Y_ts_0 = split(Xtr0,train_labels_1)
X_tr_1,Y_tr_1,X_ts_1,Y_ts_1 = split(Xtr1,train_labels_2)
X_tr_2,Y_tr_2,X_ts_2,Y_ts_2 = split(Xtr2,train_labels_3)

In [27]:
train_gram,estim_0=fit_KRR(X_tr_0,Y_tr_0,wrap_kernel_0,lam_0)

In [29]:
test_gram,predic_0=predict_KRR(X_tr_0,X_ts_0,wrap_kernel_0,estim_0)

In [ ]:
evaluate(Y_ts_0,predic_0)

In [88]:
train_gram_1,estim_1=fit_KRR(X_tr_1,Y_tr_1,wrap_kernel_1,lam_1)

Process ForkPoolWorker-41492:
Process ForkPoolWorker-41491:
Process ForkPoolWorker-41489:
Process ForkPoolWorker-41490:
Traceback (most recent call last):
Traceback (most recent call last):
  File "/opt/conda/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
Traceback (most recent call last):
Traceback (most recent call last):
  File "/opt/conda/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
  File "/opt/conda/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/opt/conda/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/opt/conda/lib/python3.7/multiprocessing/pool.py", line 121, in worker
    result = (True, func(*args, **kwds))
  File "/opt/conda/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
  File "/opt/conda/lib/python3.7/multiprocessing/process.py", line 29

KeyboardInterrupt: 

  File "/opt/conda/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
  File "/opt/conda/lib/python3.7/multiprocessing/pool.py", line 110, in worker
    task = get()
  File "/opt/conda/lib/python3.7/multiprocessing/queues.py", line 351, in get
    with self._rlock:
  File "/opt/conda/lib/python3.7/multiprocessing/pool.py", line 110, in worker
    task = get()
  File "<ipython-input-52-6eff2bbd9720>", line 8, in wrap_kernel_1
    return spectrum_combination(x1,x2,[4,6,10])
  File "/opt/conda/lib/python3.7/multiprocessing/queues.py", line 351, in get
    with self._rlock:
  File "<ipython-input-52-6eff2bbd9720>", line 2, in spectrum_combination
    return np.sum((1/len(k_list))*np.array([spectrum_kernel(x1,x2,k) for k in k_list]))
  File "/opt/conda/lib/python3.7/multiprocessing/queues.py", line 351, in get
    with self._rlock:
  File "<ipython-input-52-6eff2bbd9720>", line 2, in <listcomp>
    return np.sum((1/len(k_list))*np.array(

In [ ]:
test_gram_1,predic_1=predict_KRR(X_tr_1,X_ts_1,wrap_kernel_1,estim_1)

In [ ]:
evaluate(Y_ts_1,predic_1)

In [89]:
train_gram_2,estim_2=fit_KRR(X_tr_2,Y_tr_2,wrap_kernel_2,lam_2)

In [90]:
test_gram_2,predic_2=predict_KRR(X_tr_2,X_ts_2,wrap_kernel_2,estim_2)

In [108]:
evaluate(Y_ts_1,predic_1)

0.772